<a href="https://colab.research.google.com/github/gvogiatzis/CS4740/blob/main/CS4740_Lab_Week_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [CS4740 Labs] Lab 6: Deep Generative Models

##Introduction

One of the big successes of Deep Learning that sparked significant interest in the last few years is Deep Generative Models. These are deep neural architectures whose main goal is *analysis-by-synthesis*. In other words, they represent an attempt to analyse a complex dataset by synthesizing new datapoints that preserve the statistical properties of the original data. In achieving this, the deep generative models are able to extract meaningful information about the statistical distribution that gave rise to the original dataset, thereby providing us with some sort of *high-level understanding*. 

These Deep Generative architectures have been used to generate realistic data from a variety of sources, including among others, celebrity face images and artistic paintings!

In this lab we will explore some of the properties of Deep Generative Models by running several experiments using simple versions of popular generative architectures. 

##A very simple VAE

Let's begin by exploring a very simple toy dimensionality reduction problem. To keep things easy to visualize, we will limit ourselves to two dimensions. The code below generates the well known spiral dataset. 

In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import numpy as np

def spiral(n_points, noise=1.5):
    n = np.sqrt(np.random.rand(n_points,1)) * 780 * (2*np.pi)/360
    dx = -np.cos(n)*n + np.random.rand(n_points,1) * noise
    dy = np.sin(n)*n + np.random.rand(n_points,1) * noise
    return np.hstack((dx,dy))

Let's see what this looks like:

In [ ]:
X = spiral(n_points=1000)
plt.plot(X[:,0],X[:,1],'rx')

What we immediately observe is that while the points are two-dimensional, the space they come from is only one-dimensional. Which means this dataset could be a good candidate for a Variational Auto-Encoder with a 1-dimensional encoding. We will be using the simple MLP class below for both encoder and decoder networks. The network layers are generated according to the input of the constructor. The `hidden` holds the sizes of all the hidden layers (if any). We also define the input and output sizes.

In [ ]:
class MLPNet(nn.Module):
    def __init__(self, input_size=2, hidden=[3], output_size=2):
        super(MLPNet, self).__init__()
        s0=input_size
        self.hidden_layers = nn.ModuleList()
        for h in hidden:
            self.hidden_layers.append(nn.Sequential(nn.Linear(s0,h),nn.LeakyReLU()))
            s0=h
        self.classifier= nn.Linear(s0,output_size)
        
    def forward(self, x):
        for f in self.hidden_layers:
            x = f(x)
        return self.classifier(x)

Let us experiment with an encoder network that takes maps two dimentional input to one dimensional output:


In [ ]:
enc = MLPNet(input_size=2, hidden=[64,256,512,128], output_size=1)
X = spiral(n_points=1000)
x = torch.tensor(X,dtype=torch.float)

z = enc(x)
print(x.shape)
print(z.shape)

The decoder network will similarly map the 1D code back onto the 2D space of the original data:


In [ ]:
dec = MLPNet(input_size=1, hidden=[64,256,512,128], output_size=2)

x_=dec(z)
print(z.shape)
print(x_.shape)

We are now ready to define the training code for encoder and decoder networks. The loss function we use is MSE because the autoencoder must reproduce its own input as closely as possible. The loss we compute has this form:

`L = loss(x, dec(enc(x)))`

which penalises the difference between the input datapoint and the reconstructed version of that datapoint as it passes through the bottleneck of the autoencoder. 

Every 1000 epochs we will produce a scatterplot of the reconstructed input data superimposed on the original data. If all goes well we should observe an overlap between the two. 

In [ ]:
from itertools import chain

N = 1000
X = spiral(n_points=N)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.tensor(X,dtype=torch.float, device=device)
num_of_epochs = 10000
z_dim = 1

enc = MLPNet(input_size=2, hidden=[128,256,128], output_size=z_dim).to(device)
dec = MLPNet(input_size=z_dim, hidden=[128,256,128], output_size=2).to(device)

optimizer = optim.Adam(chain(enc.parameters(),dec.parameters()), lr=0.001)

loss = nn.MSELoss(reduction='mean')

for e in range(num_of_epochs):
    # z = enc(x)
    # # z += 1e-2*torch.randn_like(z, device=device)
    # x_ = dec(z)
    # # L = loss(x, x_) + 1e-4*z.square().sum()
    # # L = loss(x, x_)
    L = loss(x, dec(enc(x)))
    optimizer.zero_grad()
    L.backward()
    optimizer.step()

    if e%1000 ==0:
        x_ = dec(enc(x))
        x_ = x_.cpu().detach().numpy()
        plt.figure()
        plt.plot(X[:,0],X[:,1],'x')
        plt.plot(x_[:,0],x_[:,1],'+')
        plt.title(f"{L.item()}")
        plt.show()
    

The auto-encoder seems able to reproduce the input with a very simple architecture.

## A very simple GAN

Before we move to more realistic applications, let us try a very simple (perhaps the most simple) GAN solution to the same spiral dataset problem. Similar to the auto-encoder, the GAN approach also uses two networks, a generator and a discriminator. In fact the generator can be identical to the decoder network of the auto-encoder in terms of input/output dimensionality. The discriminator is slightly different to the encoder network in that its output is a binary classification probability instead of an encoding. 

Given a datapoint $x$ and some randomly generated code $z$, we would like to optimize the following function with thespect to the parameters of discriminator $D$ and generator $G$:

$$
V(D,G)=\log D(x)+\log(1-D(G(z)))
$$

Remember from lectures, that we aim to maximize $V$ with respect to $D$ and minimize it with respect to $G$. However, as explained, the term $\log(1-D(G(z)))$ has bad numerical properties as its gradient is very flat in the beggining. We therefore end up maximizing $\log D(x)+\log(1-D(G(z)))$ with respect to $D$, while also maximizing $\log(D(G(z)))$ with respect to $G$. 

The  loss function for the discriminator looks like this:

`L_dis = loss(disc(x), ones) + loss(disc(gen(z)),zeros)`

and it is a straightforward classification error for classifier network `disc`. The first term is the loss incurred by correctly classifying the real data `x` as real while the second term is the loss of correctly classifying the synthetic data `gen(z)` as synthetic.

The loss function for the generator looks like this:

`loss(disc(gen(z)),ones)`

and is essentially the mis-classification error of the discriminator classifying the synthetic datapoints `gen(z)` as real. We need to make this as big as possible, i.e. we need to make sure the generator is good enough to *fool* the discriminator.

Because we have two separate loss functions, it makes sense to have two different optimizers for each of the two networks.

In [ ]:
N = 1000

X = spiral(n_points=N)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.tensor(X,dtype=torch.float, device=device)
num_of_epochs = 50000
z_dim = 1

disc = MLPNet(input_size=2, hidden=[128,256,128], output_size=1).to(device)
gen = MLPNet(input_size=z_dim, hidden=[128,256,128], output_size=2).to(device)

optim_d = optim.Adam(disc.parameters(), lr=0.001)
optim_g = optim.Adam(gen.parameters(), lr=0.001)

loss = nn.BCEWithLogitsLoss() 

zeros = torch.zeros(N,1,device=device)
ones = torch.ones(N,1,device=device)

for e in range(num_of_epochs):
    z = torch.randn(N,z_dim, device=device) 

    L_dis = loss(disc(gen(z)),zeros) + loss(disc(x), ones)
    optim_d.zero_grad()
    L_dis.backward()
    optim_d.step()

    L_gen = loss(disc(gen(z)),ones)
    optim_g.zero_grad()
    L_gen.backward()
    optim_g.step()

    if e%1000 ==0:
        x_gen = gen(z).cpu().detach().numpy()
        plt.figure()
        plt.plot(X[:,0],X[:,1],'x')
        plt.plot(x_gen[:,0],x_gen[:,1],'+')
        plt.show()

## Auto-encoding Fashion MNIST

We will now try the same VAE architecture on Fashion MNIST, a drop-in replacement for the well known MNIST dataset but this time using images of fashion items instead of hand-written digits. Let's load it up and have a peak:

In [ ]:
dataset=torchvision.datasets.FashionMNIST('./data',download=True,transform=torchvision.transforms.ToTensor())
dataloader=torch.utils.data.DataLoader(dataset, batch_size=64)

x,_=next(iter(dataloader))
grid = torchvision.utils.make_grid(x)
plt.imshow(grid.permute([1,2,0]))
plt.show()

Since these images are the usual 28x28 grayscale, we will need to reshape them as 784-dimensional vectors, exactly as we did to the MNIST digit images a few labs back. In order to evaluate the auto-encoder network we will write a special function that plots a few images together with their reconstructions via the auto-encoder.

In [ ]:
def evaluate_autoencoder(x, enc, dec, title=""):
    x = x.view(-1,784).to(device)
    x_ = dec(enc(x))
    x = x.cpu().detach()
    x = x.view(-1,1,28,28)
    x_ = x_.cpu().detach()
    x_ = x_.view(-1,1,28,28)
    grid = torchvision.utils.make_grid(x)
    grid_ = torchvision.utils.make_grid(x_)
    plt.figure()
    f,ax = plt.subplots(1,2)
    ax[0].imshow(grid.permute([1,2,0]))
    ax[0].set_title('input')
    ax[1].imshow(grid_.permute([1,2,0]))
    ax[1].set_title('reconstruction')
    f.suptitle(title)
    plt.show()

Now we are ready to write the auto-encoder training code. It's very similar to the simple VAE code above (the same network architecture is used) but with some differences to do with image data pre-processing and dimension reshaping. We also add a noise term and a penalty for the L2 norm of the embedding that has the same effect as the KL divergence term. This brings the network closer to the true probabilistic VAE we saw in lectures.

In [ ]:
from itertools import chain

N = 1000
X = spiral(n_points=N)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.tensor(X,dtype=torch.float, device=device)
num_of_epochs = 50
z_dim = 10

enc = MLPNet(input_size=784, hidden=[128,256,128], output_size=z_dim).to(device)
dec = MLPNet(input_size=z_dim, hidden=[128,256,128], output_size=784).to(device)

optimizer = optim.Adam(chain(enc.parameters(),dec.parameters()), lr=0.001)

loss = nn.MSELoss(reduction='mean')

noise_scale =1e-2
l2penalty = 1e-3

for e in range(num_of_epochs):
    x, _ = next(iter(dataloader))
    evaluate_autoencoder(x, enc, dec, title=f"{e+1}/{num_of_epochs}")
    for x, _ in dataloader:
        x = x.to(device)
        x = x.view(-1,784)

        z = enc(x)
        z += noise_scale*torch.randn_like(z, device=device)
        x_ = dec(z)
        L = loss(x, x_) + l2penalty*z.square().sum()

        optimizer.zero_grad()
        L.backward()
        optimizer.step()

The results of the VAE look a bit fuzzy, just as we discussed in the lectures. The best theory about why this
happens is the L2 norm loss function for images. This tends to smooth out the details, putting more emphasis on global features. If you look closely, you will see that some of the detail on the clothes (t-shirt print etc) is blurred our.

Let's try interpolating between two datapoints. The idea is, we take two random images from the dataset, convert them to z-encodings and then linearly interpolate between the encodings. We then decode each of these linearly interpolated z-encodings back to image space.

In [ ]:
x0 = dataset[10][0].view(-1,784).to(device)
x1 = dataset[40][0].view(-1,784).to(device)

z0 = enc(x0)
z1 = enc(x1)
t = torch.linspace(0,1,16,device=device).view(-1,1)
z_interp=torch.lerp(z0,z1,t)
x_interp = dec(z_interp).cpu().detach().view(-1,1,28,28)

grid = torchvision.utils.make_grid(x_interp,nrow=4)
plt.figure()
plt.imshow(grid.permute([1,2,0]))
plt.show()


## Challenges:

* What can we say about the effect of the `l2penalty` and `noise_scale` term? Run a few experiments to find out.
* Can you use the code supplied in order to fit a GAN model to the Fashion MNIST data? 
* **Advanced:** If you would like to try your hand on a cutting edge research problem, think about how we could use combine VAEs with GANs? Can we have a VAE that has the standard reconstruction error, combined with an adversarial discriminator loss function? Does this undo  some of the blurriness of standard VAEs?
